# Méthodes de Recherche - cadre deterministe et observable 

Nous allons, dans ce notebook, implémenter certaines méthodes de recherche dans un cadre déterministe et observable.  

Nous avons besoin des import suivants.

In [1]:
import sys
import numpy as np
import frontiers #Module to represent frontiers of different types 
from abc import ABC, abstractmethod #To represent an abstract class
import networkx as nx #To work on graphs
from time import sleep

## Formalisation d'un problème de recherche

Nous allons commencer par formaliser la notion de problème de recherche en utilisant une classe abstraite. Nous pourrons ensuite dériver cette classe pour formaliser les différents problèmes que nous devrons résoudre.

In [2]:
"""
Abstract class to represent a search problem.  
@author: Hugo Gilbert
"""
class Problem(ABC):
    """
    An astract class to represent a search problem.
    
    Methods
    -------
    actions(state)
        Yields the set of valid actions
    transition(state, action)
        The transtion function of the search problem
    isFinal(state)
        Asserts if a state is final or not
    actionCost(state, action)
        The cost function of the search problem
    getInitialState()
        Yields the initial state of the search problem
    """
    
    @abstractmethod    
    def actions(self, state):
        """ Yields the set of valid actions.
        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        set
            a set of chars representing the valid actions in the given state     
        """

        pass
    
    
    @abstractmethod
    def transition(self, state, action):
        """ The transtion function of the search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the state obtained when performing action in state
        """
        
        pass
    

    @abstractmethod
    def isFinal(self, state):
        """ Asserts if a state is final or not.

        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        True if the state is final else False

        """
        
        pass
    
    @abstractmethod
    def actionCost(self, state, action):
        """ The cost function of the search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the cost (a positive integer) incurred when performing action in state
        """
        
        pass
    
    @abstractmethod
    def getInitialState(self):
        """ Yields the initial state of the search problem.
        Returns
        -------
        the initial state of the search problem
        """
        
        pass        

Dérivons cette classe pour représenter le problème de recherche classique de navigation en Roumanie. Nous allons pour cela représenter le réseau routier entre les différentes villes de Roumanie. 

C'est un exemple très *particulier* car dans la plupart des cas nous ne pourrons pas représenter explicitement le graphe de l'espace d'état.

In [3]:
class Romania(Problem):
    """
    A class to represent the classic navigation in 
    Romania search problem from [1].
    
    [1] Norvig, P. and Russel, S., Artificial Intelligence. A modern approach. 
    
    Methods
    -------
    actions(state)
        Yields the set of valid actions
    transition(state, action)
        The transtion function of the search problem
    isFinal(state)
        Asserts if a state is final or not
    actionCost(state, action)
        The cost function of the search problem
    getInitialState()
        Yields the initial state of the search problem
    """
    #We will here use the graph representing this problem
    #This is a very specific example, and somewhat misleading 
    #example in the sense that most of the time we cannot
    #explicitly represent the state space graph.
    G = nx.Graph()
    G.add_nodes_from(['A','B','C','D',\
                  'E','F','G','H',\
                  'I','L','M','N',\
                  'O','P','R','S',\
                  'T','U','V','Z'])

    G.add_weighted_edges_from([('A','T',118), ('A','S',140), ('A','Z',75), ('Z','O',71), ('O','S',151),\
                           ('T','L',111), ('L','M',70), ('M','D',75), ('D','C',120), ('S','F',99),\
                           ('S','R',80), ('R','C',146), ('R','P',97), ('C','P',138), ('P','B',101),\
                           ('F','B',211), ('B','G',90), ('B','U',85), ('U','H',98), ('H','E',86),\
                           ('U','V',142), ('V','I',92), ('I','N',87)])

Les états vont ici correspondre aux différentes villes identifiées chacune par un char, la première lettre de la ville. Quant aux actions, nous assimilerons l'action ''se rendre au sommet v'' avec le sommet v lui même. 

Nous allons maintenir redéfinir les méthodes une à une pour spécifier notre problème de recherche. Ces méthodes sont assez triviales:
- la méthode spécifiant les différentes actions possibles à partir d'un sommet retourne les voisins du sommet.
- la méthode de transition retourne l'action car elle est ici confondue avec le sommet vers lequel on se rend.
- la méthode testant si un état est final est un simple test vérifiant si le sommet correspond à Budapest.
- Le coût d'une action correspond au poids de l'arête correpondante. 
- la méthode retournant l'état de départ retourne le sommet correspondant à Arad.

In [4]:
class Romania(Romania):
    
    def actions(self, state):
        """ Yields the set of valid actions.
        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        set
            a set of chars representing the valid actions in the given state     
        """

        return Romania.G.neighbors(state)
    
    
    def transition(self, state, action):
        """ The transtion function of the search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the state obtained when performing action in state
        """
        return action
        
    def isFinal(self, state):
        """ Asserts if a state is final or not.

        Parameters
        ----------
        state : State
            a given state

        Returns
        -------
        True if the state is final else False

        """
        return state == 'B'
    
    def actionCost(self, state, action):
        """ The cost function of the search problem.
        Parameters
        ----------
        state : State 
            a given state  
        action : char
            a valid action in the state

        Returns
        -------
        the cost (a positive integer) incurred when performing action in state
        """
        
        return Romania.G.edges[state, action]['weight']
    
    def getInitialState(self):
        """ Yields the initial state of the search problem.
        Returns
        -------
        the initial state of the search problem
        """
        return 'A'
                

## Formalisation d'un l'arbre de recherche

Nous allons désormais définir le nœud d'un arbre de recherche qui doit contenir un état mais aussi des informations sur le nœud de recherche parent et le coût du chemin de l'état initial jusqu'à lui. 

In [5]:
class Node:
    """
    A class to define a node in the search tree. 
    
    Attributes
    ----------
    state : State
        A state of the search problem
    problem : Problem
        The search problem to be solved
    fatherNode : Node
        The father node in the search tree
    actionFromFather : char
        The action that led to this new node from the father node
    pathCost : int
        The cost of the path from the root to this node in the search tree
        
    Methods
    -------
    expand()
        Method to expand the node
    getSolution()
        Method to obtain the list of actions from the root to a terminal node 
        
    """
    def __init__(self, problem, state, fatherNode, actionFromFather):
        """
        Parameters
        ----------
        problem : Problem
            The search problem to be solved
        state : State
            A state of the search problem
        fatherNode : Node
            The father node in the search tree
        actionFromFather : char
            The action that led to this new node from the father node
        """
        self.state = state
        self.problem = problem 
        self.fatherNode = fatherNode
        self.actionFromFather = actionFromFather
        self.pathCost = 0
        if(fatherNode):
            self.pathCost = fatherNode.pathCost + problem.actionCost(fatherNode.state, actionFromFather) 
    
    def __str__(self):
        return str(self.state)
    
    def __lt__(self,other):
        return self.state < other.state

Définissons maintenant la méthode permettant d'étendre un nœud, pour produire les nœuds fils.

In [6]:
class Node(Node):
    def expand(self):
        """ Method to expand the node. 
        Returns
        -------
        res : list
            The list of succesor nodes obtained from expanding the current node.
        """
        res = []
        for action in self.problem.actions(self.state):
            nextState = self.problem.transition(self.state,action)
            res.append(Node(self.problem, nextState, self, action))
        return res

Enfin, une fois la solution jusqu'au nœud terminal trouvé, il faut remonter l'arbre de recherche afin de pouvoir renvoyer l'ensemble du chemin de l'état initial jusqu'à l'état final.

In [7]:
class Node(Node):
    def getSolution(self):
        """ Method to obtain the list of actions from the root to a terminal node. 
        
        Returns
        -------
        res : []
            List of actions from the root to the terminal node.
        """
        res = []
        node = self
        while(node.fatherNode):
            res.append(node.actionFromFather)
            node = node.fatherNode
        res.reverse()
        return res

## Méthode de recherche en largeur

Nous allons désormais implémenter et tester l'algorithme de recherche par parcours en largeur. Cet algorithme est de type graph-search, i.e., il utilise une frontière et un ensemble des nœuds explorés. La frontière est de type FIFO.

In [8]:
class BreadthFirst:
    """ Class for the breadth first search algorithm.
    This is a graph-search type algorithm with a FIFO-type frontier.
    """
    
    def __init__(self, problem):
        """
        Parameters
        ----------
        problem : Problem
            The search problem to be solved.
        """
        self.problem = problem
        
    def solve(self):
        """
        runs the BreadthFirst algorithm
        """
        node = Node(self.problem, self.problem.getInitialState(), None, None)
        if(self.problem.isFinal(node)):
                return node.getSolution()
        self.exploredSet = set()
        self.frontier = frontiers.FrontierFIFO()
        self.frontier.push(node)
        while(True): 
            if(self.frontier.isEmpty()):
                raise Exception('problem cannot be solved')
            print(self.frontier)
            sleep(1)
            print("-------\n")
            node = self.frontier.pop() 
            self.exploredSet.add(node.state)
            for succ_node in node.expand():
                if succ_node.state in self.exploredSet:
                    continue
                if(self.problem.isFinal(succ_node.state)):
                    return succ_node.getSolution()
                self.frontier.push(succ_node)

Testons désormais notre méthode sur notre problème de navigation en Roumanie.

In [9]:
romania = Romania()
bf = BreadthFirst(romania)
print(bf.solve())

A


-------

Z S T
-------

L Z S
-------

R F O L Z
-------

R F O L
-------

M R F O
-------

M R F
-------

['S', 'F', 'B']


## Méthode de recherche en profondeur

Nous allons désormais implémenter et tester l'algorithme de recherche par parcours en profondeur. Cet algorithme est de type *tree-search*, i.e., il n'utilise pas un ensemble des nœuds explorés. La frontière est de type LIFO. 

Pour éviter que notre algorithme ne génère des boucles infinies, nous allons implémenter une méthode permettant de voir si un nœud a déjà été vu sur le chemin actuellement parcouru. 

In [10]:
class Node(Node):
    def generate_loop(self):
        node = self
        while(node.fatherNode):
            other = node.fatherNode
            if(self.state == other.state):
                return True
            node = other
        return False

In [11]:
class DepthFirst:
    """ Class for the Depth First Search Algorithm.
    """
    
    def __init__(self, problem):
        """
        Parameters
        ----------
        problem : Problem
            The search problem to be solved.
        """
        self.problem = problem
        
    def solve(self):
        """
        runs the depth-first search algorithm
        """
        node = Node(self.problem, self.problem.getInitialState(), None, None)
        if(self.problem.isFinal(node)):
            return node.getSolution()
        self.frontier = frontiers.FrontierLIFO()
        self.frontier.push(node)
        while(True): 
            if(self.frontier.isEmpty()):
                raise Exception('problem cannot be solved')
            print(self.frontier)
            sleep(1)
            print("-------\n")
            node = self.frontier.pop() 
            for succ_node in node.expand():
                if succ_node.generate_loop():
                    continue
                if(self.problem.isFinal(succ_node.state)):
                    return succ_node.getSolution()
                self.frontier.push(succ_node)

Testons désormais notre méthode sur notre problème de navigation en Roumanie.

In [12]:
df = DepthFirst(romania)
print(df.solve())

A


-------

T S Z
-------

T S O
-------

T S S
-------

T S F R
-------

T S F C P
-------

['Z', 'O', 'S', 'R', 'P', 'B']


## Méthode de recherche A*

Pour terminer sur la méthode A*, nous devons rajouter une heuristique à notre problème de navigation, par exemple la distance à vol d'oiseau entre deux villes.

In [13]:
class Romania(Romania):
    st_line = {'A':366,'B':0,'C':160,'D': 242, 'E':161,\
              'F':176, 'G':77, 'H':151, 'I': 226, 'L':244,\
              'M':241,'N':234,'O':380, 'P':100, 'R':193,\
              'S':253,'T':329,'U':80,'V':199,'Z':374}
    
    def heuristic(self, state):
        """returns a value estimating (lower bound) the distance 
        between the current city and Budapest using the straight-line distance.
        
        Returns
        -------
        int 
            straight-line distance between the city corresponding to the state and Budapest
        """
        return Romania.st_line[state]

Nous allons désormais implémenter et tester l'algorithme de recherche A*. Cet algorithme est de type graph-search, i.e., il utilise une frontière et un ensemble des nœuds explorés. La frontière est une file de priorité, avec une priorité correspondant à la somme du coût du nœud et de sa valeur heuristique.

In [14]:
class AStar:
    """ Class for the AStar Algorithm.
    """
    
    def __init__(self, problem, heuristic):
        """
        Parameters
        ----------
        problem : Problem
            The search problem to be solved.
        heuristic : function
            The heuristic function to guide the search.
        """
        self.problem = problem
        self.heuristic = heuristic
    
    def solve(self):
        """
        runs the AStar algorithm
        """
        self.exploredSet = set()
        self.frontier = frontiers.Frontier()
        self.frontier.push(self.heuristic(self.problem.getInitialState()),Node(self.problem, self.problem.getInitialState(), None, None))
        while(True): 
            if(self.frontier.isEmpty()):
                raise Exception('problem cannot be solved')
            print(self.frontier)
            sleep(1)
            print("-------\n")
            (score, node) = self.frontier.pop() 
            if(self.problem.isFinal(node.state)):
                return node.getSolution()
            self.exploredSet.add(node.state)
            for succ_node in node.expand():
                if succ_node.state in self.exploredSet:
                    continue
                self.frontier.push(succ_node.pathCost + self.heuristic(succ_node.state), succ_node)

Testons désormais l'algorithme A*.

In [15]:
romania = Romania()
astar = AStar(romania,romania.heuristic)
    
print(astar.solve())

366A


-------

393S 447T 449Z
-------

413R 415F 671O 449Z 447T
-------

415F 447T 417P 449Z 526C 671O
-------

417P 447T 450B 449Z 526C 671O
-------

418B 447T 671O 526C 449Z
-------

['S', 'R', 'P', 'B']
